In [ ]:
%cd C:/DIEM/recommender_module/

In [2]:
import pandas as pd
import numpy as np
import joblib

from utils.resources_functions import finding_ids_with_loop, act_with_res, res_freq, filter_res, find_normal_res
from utils.dice_functions import dice_recommendations

# Suppress all warnings
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2

In [3]:
data_folder = 'bpi17'

In [4]:
train_data = pd.read_csv("./" + data_folder + "/train_data.csv")
test_data = pd.read_csv("./" + data_folder + "/test_log_only_last_act_with_tsplit.csv")
test_log = pd.read_csv("./" + data_folder + "/test_log_with_tsplit.csv")

In [5]:
# del train_data["start_timestamp"]
# del train_data["end_timestamp"]
# del test_data["start_timestamp"]
# del test_data["end_timestamp"]
# del test_log["start_timestamp"]
# del test_log["end_timestamp"]

In [6]:
# Loading model
predictive_model = joblib.load("./" + data_folder + "/catboost_time.joblib")

In [7]:
# # For BAC
# case_id_name = 'REQUEST_ID'
# activity_column_name = 'ACTIVITY'
# resource_column_name = 'CE_UO'
# start_date_name = "start:timestamp"
# end_date_name = "time:timestamp"
# outcome_name = "leadtime"

# For BPI17
case_id_name = "case:concept:name"
activity_column_name = "concept:name"
resource_column_name = "org:resource"
start_date_name = "start:timestamp_"
end_date_name = "time:timestamp_"
outcome_name = "leadtime"

In [ ]:
print("Number of resources in train data:",len(train_data[resource_column_name].unique()))

In [ ]:
# List of activities
list_activities = list(train_data[activity_column_name].unique())
print("Number of activities:", len(list_activities))

In [10]:
# # For BAC
# continuous_features = ['time_from_start', 'time_from_previous_event(start)', '# ACTIVITY=Service closure Request with network responsibility',
#                        "activity_duration",
#                        '# ACTIVITY=Service closure Request with BO responsibility', '# ACTIVITY=Pending Request for Reservation Closure', '# ACTIVITY=Pending Liquidation Request',
#                        '# ACTIVITY=Request completed with account closure', '# ACTIVITY=Request created', '# ACTIVITY=Authorization Requested',
#                        '# ACTIVITY=Evaluating Request (NO registered letter)', '# ACTIVITY=Network Adjustment Requested', '# ACTIVITY=Pending Request for acquittance of heirs',
#                        '# ACTIVITY=Request deleted', '# ACTIVITY=Back-Office Adjustment Requested', '# ACTIVITY=Evaluating Request (WITH registered letter)',
#                        '# ACTIVITY=Request completed with customer recovery', '# ACTIVITY=Pending Request for Network Information']

# # For BPI17
continuous_features = ['time_from_start',
       'time_from_previous_event(start)', 'time_from_midnight', 
       'event_duration',  '# ACTIVITY=O_Cancelled',
       '# ACTIVITY=O_Created', '# ACTIVITY=O_Sent (mail and online)',
       '# ACTIVITY=O_Sent (online only)', '# ACTIVITY=A_Submitted',
       '# ACTIVITY=A_Concept', '# ACTIVITY=A_Incomplete',

       '# ACTIVITY=A_Cancelled', '# ACTIVITY=A_Denied',
       '# ACTIVITY=A_Accepted', '# ACTIVITY=O_Returned',
       '# ACTIVITY=A_Validating', '# ACTIVITY=A_Create Application',
       '# ACTIVITY=O_Accepted', '# ACTIVITY=O_Create Offer',
       '# ACTIVITY=A_Complete']

In [11]:
result_df = pd.DataFrame() # Empty dataframe for storing results

In [12]:
# test_ = test_log.iloc[:5]

In [ ]:
rec_df_ = dice_recommendations(train_data, test_log, test_data, 
                              threshold = 0.99, 
                              predictive_model = predictive_model, 
                              continuous_features = continuous_features, 
                              dice_method = "random", 
                              REDUCED_KPI_TIME = 0.5, TOTAL_CFS = 100, 
                              result_df = result_df, 
                              WINDOW_SIZE = 3, 
                              cols_to_vary = ["NEXT_ACTIVITY", "NEXT_RESOURCE"], 
                              case_id_name = case_id_name, 
                              activity_column_name = activity_column_name, 
                              resource_column_name = resource_column_name, 
                              outcome_name = outcome_name,
                              start_date_name = start_date_name, 
                              end_date_name = end_date_name,
                              time_limit = 50*60)

In [ ]:
rec_df_

In [ ]:
rec_df_["valid_cfes"].value_counts()

In [16]:
# rec_df_.to_csv("./bpi17_results_05_099_3.csv", index=False)